<a href="https://colab.research.google.com/github/HSE-LAMBDA/MLatFIAN2020/blob/master/seminar04/MLatFIAN2020_seminar04_homework.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
import numpy as np
import matplotlib.pyplot as plt

## Task 1

(Titanic data again)

Build a model with `sklearn`'s `LogisticRegression` or `SVC` to get the accuracy of at least 0.81 on the test set. Can you get higher? 0.84?

Some (optional) suggestions:
- Add new features (e.g. missing value indicator columns)
- Fill missing values
- Encode categorical features (e.g. one-hot encoding)
- Scale the features (e.g. with standard or robust scaler)
- Think of other ways of preprocessing the features (e.g. `Fare` $\to$ `log(Fare)`)
- Try adding polynomial features
- use `sklearn.model_selection.GridSearchCV` to search for the best hyperparameters (https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.GridSearchCV.html)



In [ ]:
!wget https://github.com/HSE-LAMBDA/MLDM-2020/raw/master/day-01/train.csv

In [182]:
import pandas as pd
data = pd.read_csv("train.csv", index_col='PassengerId')
data.head()

,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
PassengerId,,,,,,,,,,,
1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


#### About the data
Here's some of the columns
* Name - a string with person's full name
* Survived - 1 if a person survived the shipwreck, 0 otherwise.
* Pclass - passenger class. Pclass == 3 is cheap'n'cheerful, Pclass == 1 is for moneybags.
* Sex - a person's gender
* Age - age in years, if available
* SibSp - number of siblings on a ship
* Parch - number of parents on a ship
* Fare - ticket cost
* Embarked - port where the passenger embarked
 * C = Cherbourg; Q = Queenstown; S = Southampton

In [276]:
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.pipeline import make_pipeline
from sklearn.compose import make_column_transformer
from sklearn.preprocessing import StandardScaler, RobustScaler, PolynomialFeatures, OneHotEncoder
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split

In [347]:
def feature_selection_and_preprocessing(dataset):
    #Подготовка данных
#     features = dataset[["Age", "Fare", "SibSp", "Parch", "Sex", "Pclass",'Embarked']].copy()
    features = dataset[["Age", "SibSp", "Parch", "Sex", "Pclass"]].copy()
    #Заполняем пустые значения для возраста средним значением для каждого Sex
    features.loc[features['Sex'] ==
         'female'] = features.loc[features['Sex'] == 
                              'female'].fillna(value=features.loc[features['Sex'] == 
                                                              'female']['Age'].mean())
    features.loc[features['Sex'] ==
             'male'] = features.loc[features['Sex'] == 
                                  'male'].fillna(value=features.loc[features['Sex'] == 
                                                              'male']['Age'].mean())
#     #Меняем 
    features.loc[features['Sex'] == 'male', 'Sex'] = 1
    features.loc[features['Sex'] == 'female', 'Sex'] = -1

#     #Заполняем нулевые значения Fare средним значением для каждого Pclass (Здесь только 'male')
#     features.loc[(features['Pclass'] ==
#              1) & (features['Fare'] == 0.0)] = features.loc[(features['Pclass'] ==
#                                                            1) & (features['Fare'] != 0.0)]['Fare'].mean()
#     features.loc[(features['Pclass'] ==
#              2) & (features['Fare'] == 0.0)] = features.loc[(features['Pclass'] ==
#                                                            2) & (features['Fare'] != 0.0)]['Fare'].mean()
#     features.loc[(features['Pclass'] ==
#              3) & (features['Fare'] == 0.0)] = features.loc[(features['Pclass'] ==
#                                                            3) & (features['Fare'] != 0.0)]['Fare'].mean()

# #     #Пременяем функциональную зависимость к Fare
#     features['Fare'] = np.log(features['Fare'])
    
    features['Age'] = features['Age'] / features['Age'].max()
#     features['Embarked'] = data.Embarked.fillna('unknown') 
#     В Embarked не верю
    
    
    return features

In [348]:
data = pd.read_csv("train.csv", index_col='PassengerId')
data_train, data_test = train_test_split(data, test_size=200, random_state=42)

feature_selection_and_preprocessing(
        data_train.drop('Survived', axis=1)
    )

,Age,SibSp,Parch,Sex,Pclass
PassengerId,,,,,
327,0.762500,0,0,1,3
535,0.375000,0,0,-1,3
536,0.087500,0,2,-1,2
484,0.787500,0,0,-1,3
763,0.250000,0,0,1,3
...,...,...,...,...,...
107,0.262500,0,0,-1,3
271,0.380961,0,0,1,1
861,0.512500,2,0,1,3


In [349]:
model = make_pipeline(
    ###
    
#     make_column_transformer(
#         (OneHotEncoder(sparse=False), ['Embarked']),
#         remainder='passthrough'
#     ),
    
    PolynomialFeatures(2),
    RobustScaler(),
    LogisticRegression()
)


# Validation code (do not touch)
data = pd.read_csv("train.csv", index_col='PassengerId')
data_train, data_test = train_test_split(data, test_size=200, random_state=42)

model.fit(
    feature_selection_and_preprocessing(
        data_train.drop('Survived', axis=1)
    ),
    data_train['Survived']
)

train_predictions = model.predict(
    feature_selection_and_preprocessing(
        data_train.drop('Survived', axis=1)
    )
)

test_predictions = model.predict(
    feature_selection_and_preprocessing(
        data_test.drop('Survived', axis=1)
    )
)

print("Train accuracy:", accuracy_score(
    data_train['Survived'],
    train_predictions
))
print("Test accuracy:", accuracy_score(
    data_test['Survived'],
    test_predictions
))

Train accuracy: 0.8263386396526773
Test accuracy: 0.825
